# MNIST Dataset Example

This Jupyter Notebook demonstrates how to build and use a neural network with convolutional and pooling layers to process and classify images from the MNIST dataset.

## Download the Dataset

To begin, you need to download the `mnist.npz` dataset file into your working directory. This file contains pre-split training and testing sets of the MNIST dataset. You can download it from the following link:

[Download MNIST Dataset](https://s3.amazonaws.com/img-datasets/mnist.npz)

Ensure that the downloaded file is placed in the same directory as this notebook to load the dataset seamlessly.

In [ ]:
import numpy as np

from src import NeuralNetwork

from src.loss import CategoricalCrossentropy
from src.optimizer import Adam
from src.scheduler import CosineScheduler
from src.config import FeedForwardConfig, TrainingConfig
from src.structure import Dense, Convolution, Dropout, MaxPool, Flatten, LeakyRelu, Softmax
from src.encode import OneHotEncoder
from src.preprocessing import min_max_scaler

In [ ]:
path = './mnist.npz'

with np.load(path, allow_pickle=True) as f:
    x_train, y_train = f["x_train"], f["y_train"]
    x_test, y_test = f["x_test"], f["y_test"]

x_train = min_max_scaler(np.array(x_train), 0, 1)
x_test = min_max_scaler(np.array(x_test), 0, 1)

classes = tuple(np.unique(y_train))

In [ ]:
config = FeedForwardConfig(
    network_structure= [
        Convolution(channels=8, kernel_shape=(3, 3), padding=1),
        LeakyRelu(),
        MaxPool(channels=8, filter_shape=(2, 2)),
        Convolution(channels=16, kernel_shape=(3, 3), padding=1),
        LeakyRelu(),
        MaxPool(channels=16, filter_shape=(2, 2)),
        Convolution(channels=32, kernel_shape=(3, 3), padding=1),
        LeakyRelu(),
        Flatten(),
        Dense(64),
        LeakyRelu(),
        Dropout(p=0.1),
        Dense(10),
        Softmax(),
    ],
    classes=classes,
    encoder=OneHotEncoder,
)

nn = NeuralNetwork(config)

nn.train(
    x_train,
    y_train,
    x_test,
    y_test,
    config=TrainingConfig(
        lr=CosineScheduler(learning_rate=0.0001, min_lr=1e-7, max_steps=100),
        patience_stop=100,
        loss=CategoricalCrossentropy(),
        optimizer=Adam(),
        batch_size=32,
        debug=True,
        store=None,
    ),
)


print(f"test: {nn.evaluate(x_test, y_test)}, train: {nn.evaluate(x_train, y_train)}")